# **Chapter 5: Formatting Output & Speaking for Llama3**

---
**Lesson:**

Llama3 is designed to offer versatile output formatting capabilities, tailored to meet your needs. Among these capabilities is the use of XML tags to delineate responses from any extraneous text. You've already discovered how XML tags can enhance the clarity and structure of your prompts for Llama3. Similarly, you can leverage XML tags to make Llama3's outputs more transparent and comprehensible for human readers.

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

modelId = 'meta.llama3-8b-instruct-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

def invoke_model_and_get_response(prompt_data): 
    body = json.dumps({ 
        'prompt': prompt_data,
        'max_gen_len': 512,
        'top_p': 0.9,
        'temperature': 0.2
    })

    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
        response_body = json.loads(response.get('body').read().decode('utf-8'))
        outputText = response_body['generation'].strip()
        return outputText

    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'AccessDeniedException':
            return (f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubleshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        else:
            raise

# **Examples:**

**Example 5.1 - Tagging**

It turns out we can also achieve a similar outcome by telling Llama3 to put the poem in XML tags.

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template=""" 
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    Please write a haiku about {ANIMAL}. Put it in <haiku> tags.
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """
)

prompt = multi_var_prompt.format(ANIMAL="Rabbit")

response = invoke_model_and_get_response(prompt)
print(response)

**Example 5.2 - Tagging #2**

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template=""" 
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    Please write a haiku about {ANIMAL}. Put it in <haiku> tags.
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """
)

prompt = multi_var_prompt.format(ANIMAL="Cat")

response = invoke_model_and_get_response(prompt)
print(response)

**Example 5.3 - JSON**

Llama3 also excels at using other output formatting styles, notably JSON.

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template=""" 
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    Please write a haiku about {ANIMAL}. Use JSON format with the keys as "first_line", "second_line", and "third_line".
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """
)

prompt = multi_var_prompt.format(ANIMAL="Cat")

response = invoke_model_and_get_response(prompt)
print(response)

**Example 5.4 - Combination**

Below is an example of multiple input variables in the same prompt AND output formatting specification, all done using XML tags.

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL", "ADJECTIVE"], 
    template=""" 
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    Hey. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags.
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """
)

prompt = multi_var_prompt.format(EMAIL="Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.", ADJECTIVE="Olde English")

response = invoke_model_and_get_response(prompt)
print(response)

# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 5.1 - Steph Curry GOAT**

Forced to make a choice, Llama3 designates Michael Jordan as the best basketball player of all time. Can we get Llama3 to pick someone else?
Modify the "best basketball player" prompt below and use the "speaking for Llama3" technique to make Llama3 make a detailed argument that the best basketball player of all time is Stephen Curry.

In [ ]:
# Create the prompt
prompt_data = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
<|eot_id|>
    
<|start_header_id|>user<|end_header_id|>
Please give me reasoning as to why Stephen Curry is the GOAT.
<|eot_id|>
    
<|start_header_id|>assistant<|end_header_id|>
"""

response = invoke_model_and_get_response(prompt_data)
print(response)

**Exercise 5.2 - Two Haikus**

Modify the haiku prompt in the highlighted template box below and use XML tags so that Llama3 writes two haikus about the animal instead of just one. It should be clear where one poem ends and the other begins.

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template=""" 
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    Please write two haikus about {ANIMAL}. Put it in <haiku> tags.
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """
)

prompt = multi_var_prompt.format(ANIMAL="Cat")

response = invoke_model_and_get_response(prompt)
print(response)

**Exercise 5.3 - Two Haikus, Two Animals**

Modify the haiku prompt in the cell below so that Llama3 produces two haikus about two different animals. 
Use {ANIMAL1} as a stand-in for the first substitution, and {ANIMAL2} as a stand-in for the second substitution.

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL1", "ANIMAL2"], 
    template=""" 
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful, respectful, and honest assistant. Always answer as honest as possible, while being safe.
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    Please write two different haikus. One about {ANIMAL1} and one about {ANIMAL2}. Put them in <haiku> tags.
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """
)

prompt = multi_var_prompt.format(ANIMAL1="Cat",ANIMAL2="Dog")

response = invoke_model_and_get_response(prompt)
print(response)

# Chapter 5 - END.